In [25]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm

In [26]:
class BrainTumorDataset(Dataset):
    def __init__(self, data_dir, transform=False):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
    def classes(self):
        return self.data.classes

In [ ]:
data_dir = "brain_tumor_dataset"

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

brain_tumor_dataset = BrainTumorDataset(data_dir, transform)

train_size = int(0.8 * len(brain_tumor_dataset))
test_size = len(brain_tumor_dataset) - train_size

train_braintumor_dataset, test_braintumor_dataset = random_split(brain_tumor_dataset, [train_size, test_size])

In [28]:
brain_train_tumor_dataloader = DataLoader(
    train_braintumor_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0
)

brain_test_tumor_dataloader = DataLoader(
    test_braintumor_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)

In [29]:
classes = brain_tumor_dataset.classes()

In [30]:
class BrainTumorNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)


        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.5)
    def Convolution(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
    
        return x
    
    def forward(self, x):
        x = self.Convolution(x)
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x


In [31]:
import torch.nn as nn
import torch.nn.functional as F

class BrainTumorNeuralNet2(nn.Module):
    def __init__(self):
        super().__init__()
        # Use BatchNorm2d after every Conv layer to help the model learn faster
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        
        # Adaptive pooling ensures the input to the Linear layer is always 2x2
        # no matter what size the original image was.
        self.adaptive_pool = nn.AdaptiveAvgPool2d((2, 2))

        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # We group convs into blocks and pool after every two layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) # Pool 1
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # Pool 2
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(F.relu(self.bn6(self.conv6(x)))) # Pool 3
        
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

In [32]:
net = BrainTumorNeuralNet2()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

Training epoch 0 ... 


Training the Network:   0%|          | 0/7 [00:00<?, ?it/s]

Loss 0.0223
Training epoch 1 ... 


Training the Network:   0%|          | 0/7 [00:00<?, ?it/s]

Loss 0.0217
Training epoch 2 ... 


...

Loss 0.0000
Training epoch 48 ... 


Training the Network:   0%|          | 0/7 [00:00<?, ?it/s]

Loss 0.0000
Training epoch 49 ... 


Training the Network:   0%|          | 0/7 [00:00<?, ?it/s]

Loss 0.0001


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

net.train()
for epoch in range(50):
    
    print(f"Training epoch {epoch} ... ")

    running_loss = 0.0
    for inputs, labels in tqdm(brain_train_tumor_dataloader, desc="Training the Network"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad(set_to_none=True)
        output = net(inputs)

        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Loss {running_loss / len(train_braintumor_dataset):.4f}")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BrainTumorNeuralNet3(nn.Module):
    def __init__(self):
        super().__init__()
        # Use BatchNorm2d after every Conv layer to help the model learn faster
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        
        # Adaptive pooling ensures the input to the Linear layer is always 2x2
        # no matter what size the original image was.
        self.adaptive_pool = nn.AdaptiveAvgPool2d((2, 2))

        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # We group convs into blocks and pool after every two layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) # Pool 1
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # Pool 2
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(F.relu(self.bn6(self.conv6(x)))) # Pool 3
        
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

Testing the model:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy = 78.43137254901961%


In [ ]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for inputs, labels in tqdm(brain_test_tumor_dataloader, desc="Testing the model"):
        output = net(inputs)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total 
print(f"Accuracy = {accuracy}%")

In [35]:
torch.save(net.state_dict(), 'trained_braintumor_net.pth')

In [36]:
net = BrainTumorNeuralNet()
net.load_state_dict(torch.load('trained_braintumor_net.pth'))

RuntimeError: Error(s) in loading state_dict for BrainTumorNeuralNet:
	Unexpected key(s) in state_dict: "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn2.num_batches_tracked", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var", "bn3.num_batches_tracked", "bn4.weight", "bn4.bias", "bn4.running_mean", "bn4.running_var", "bn4.num_batches_tracked", "bn5.weight", "bn5.bias", "bn5.running_mean", "bn5.running_var", "bn5.num_batches_tracked", "bn6.weight", "bn6.bias", "bn6.running_mean", "bn6.running_var", "bn6.num_batches_tracked". 

In [ ]:
def load_image(image_path):
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ["/Users/nathandesousa/Desktop/Projects/Ai/Image Classification/Brain Tumor/brain_tumor_dataset/yes/Y77.jpg"]
#https://www.cancer.gov/news-events/cancer-currents-blog/2018/liquid-biopsy-childhood-brain-tumors
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Predication: {classes[predicted.item()]}')

Predication: yes
